In [1]:
using Pkg

In [2]:
Pkg.status()

    Status `~/.julia/environments/v1.0/Project.toml`
  [8f4d0f93] Conda v1.1.1
  [7073ff75] IJulia v1.14.1
  [4076af6c] JuMP v0.18.5+ #master (https://github.com/JuliaOpt/JuMP.jl.git)
  [4f5ebc0c] Linda v0.0.0 [`../../../../../mnt/c/Users/mathi/GitHub/Linda.jl`]
  [fdba3010] MathProgBase v0.7.7
  [6405355b] Mosek v0.9.6
  [438e738f] PyCall v1.18.5
  [293ab2c4] SDP v0.1.0 [`../../../../../mnt/c/Users/mathi/GitHub/SDP.jl`]
  [73c0e4d2] Tulip v0.0.0 [`../../../../../mnt/c/Users/mathi/GitHub/Tulip.jl`]


In [3]:
Pkg.activate(".")
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %

In [4]:
Pkg.status()

Project JuMP-tuto v0.0.0
    Status `/mnt/c/Users/mathi/GitHub/tutorial_jump/Project.toml`
  [9961bab8] + Cbc v0.5.2
  [60bf3e95]   GLPK v0.9.1
  [7073ff75] + IJulia v1.16.0
  [4076af6c] ↑ JuMP v0.18.5 ⇒ v0.18.5+ #master (https://github.com/JuliaOpt/JuMP.jl.git)
  [91a5bcdd]   Plots v0.23.0
  [de0858da]   Printf 
  [9a3f8284] + Random 
  [8dfed614]   Test 


In [5]:
using JuMP
const MOI = JuMP.MathOptInterface

MathOptInterface

In [6]:
using GLPK
using Cbc

# Facility location problem

Benchmark instances: http://resources.mpi-inf.mpg.de/departments/d1/projects/benchmarks/UflLib/

## Uncapacitated facility location

### Problem description

* $M=\{1, \dots, m\}$ clients, $N=\{ 1, \dots, n\}$ sites where a facility can be built.
* $f_j$: fixed cost of building a facility at site $j$
* $c_{i, j}$: cost for serving customer $i$ from facility $j$

### Compact MILP formulation

$$
    \begin{array}{cl}
        \min_{x, y} \ \ \ &
            \sum_{i, j} c_{i, j} x_{i, j} + 
            \sum_{j} f_{j} y_{j}\\
        s.t. &
            \sum_{j} x_{i, j} = 1, \ \ \ \forall i \in M\\
        &   \sum_{i} x_{i, j} \leq m y_{j}, \ \ \ \forall j \in N\\
        &   x_{i, j}, y_{j} \in \{0, 1\}, \ \ \ \forall i \in M, j \in N
    \end{array}
$$

In [7]:
using Random

In [8]:
n = 30  # number of facility locations
m = 80  # number of clients

80

In [9]:
# Draw costs
Random.seed!(0)
f = rand(1000:10000, n);
c = rand(0:1000, m, n);

In [40]:
model = Model(with_optimizer(GLPK.Optimizer, msg_lev=3))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [41]:
model_ = direct_model(model.moi_backend)
model = model_

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [42]:
backend(model)

MathOptInterface.Utilities.CachingOptimizer{MathOptInterface.AbstractOptimizer,MathOptInterface.Utilities.UniversalFallback{JuMPMOIModel{Float64}}}(MathOptInterface.Bridges.LazyBridgeOptimizer{GLPK.Optimizer,MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}}(A LinQuadOptInterface model with backend:
Prob(Ptr{Nothing} @0x0000000006b18c40), MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}(MathOptInterface.Bridges.AllBridgedConstraints{Float64}("", false, FEASIBILITY_SENSE::OptimizationSense = 2, false, MathOptInterface.ScalarAffineFunction{Float64}(MathOptInterface.ScalarAffineTerm{Float64}[], 0.0), 0, nothing, Dict{MathOptInterface.VariableIndex,String}(), nothing, 0, Dict{MathOptInterface.ConstraintIndex,String}(), nothing, Int64[], MathOptInterface.Bridges.AllBridgedConstraintsScalarConstraints{Float64,MathOptInterface.SingleVariable}(Tuple{MathOptInterface.ConstraintIndex{MathOptI

In [10]:
model = Model(with_optimizer(Cbc.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: SolverName() attribute not implemented by the optimizer.

In [43]:
# Create y variables
@variable(model, y[1:m], Bin);
@variable(model, x[1:m, 1:n], Bin);

In [44]:
# set objective
fixed_cost = sum([f[j]*y[j] for j in 1:n])
expr = sum([c[i, j]*x[i, j] for i in 1:m for j in 1:n])

@objective(model, Min, fixed_cost + expr);

In [45]:
# Add constraints

# Each client is serve exactly once
@constraint(model, [i in 1:m],
    sum(x[i, j] for j in 1:n) == 1
)

# Fixed cost of opening facilities
@constraint(model, [j in 1:n],
    sum(x[i, j] for i in 1:m) <= m*y[j]
);

In [46]:
optimize!(model)

GLPK Simplex Optimizer, v4.64
110 rows, 2480 columns, 4830 non-zeros
      0: obj =   0.000000000e+00 inf =   8.000e+01 (80)
    110: obj =   4.418300000e+04 inf =   0.000e+00 (0)
*   414: obj =   7.002150000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Integer Optimizer, v4.64
110 rows, 2480 columns, 4830 non-zeros
2480 integer variables, all of which are binary
Integer optimization begins...
+   414: mip =     not found yet >=              -inf        (1; 0)
+   602: >>>>>   3.415900000e+04 >=   8.612000000e+03  74.8% (31; 0)
+   768: >>>>>   3.230000000e+04 >=   8.656000000e+03  73.2% (52; 1)
+  1556: >>>>>   3.068500000e+04 >=   8.837000000e+03  71.2% (167; 6)
+  1978: >>>>>   2.815000000e+04 >=   9.656000000e+03  65.7% (228; 15)
+  2214: >>>>>   2.606300000e+04 >=   9.858000000e+03  62.2% (260; 31)
+  4619: >>>>>   2.509200000e+04 >=   1.103600000e+04  56.0% (574; 73)
+  7947: >>>>>   2.495000000e+04 >=   1.291800000e+04  48.2% (987; 138)
+  9969: >>>>>   2.451100000e+

In [48]:
mode(model)

AUTOMATIC::ModelMode = 0

### Extended MILP formulation

$$
    \begin{array}{cl}
        \min_{x, y} \ \ \ &
            \sum_{i, j} c_{i, j} x_{i, j} + 
            \sum_{j} f_{j} y_{j}\\
        s.t. &
            \sum_{j} x_{i, j} = 1, \ \ \ \forall i \in M\\
        &   x_{i, j} \leq y_{j}, \ \ \ \forall i \in M, j \in N\\
        &   x_{i, j}, y_{j} \in \{0, 1\}, \ \ \ \forall i \in M, j \in N
    \end{array}
$$

In [40]:
model_ext = Model(with_optimizer(Cbc.Optimizer, XX=0))

MethodError: MethodError: no method matching Cbc.Optimizer(; XX=0)
Closest candidates are:
  Cbc.Optimizer() at /home/mtanneau/.julia/packages/Cbc/Jx1MC/src/MOI_wrapper.jl:11 got unsupported keyword argument "XX"

In [34]:
# Create y variables
@variable(model_ext, y[1:m], Bin);
@variable(model_ext, x[1:m, 1:n], Bin);

In [35]:
# set objective
fixed_cost = sum([f[j]*y[j] for j in 1:n])
expr = sum([c[i, j]*x[i, j] for i in 1:m for j in 1:n])
                
@objective(model_ext, Min, fixed_cost + expr);

In [36]:
# Add constraints

# Each client is serve exactly once
@constraint(model_ext, [i in 1:m],
    sum(x[i, j] for j in 1:n) == 1
)

# Fixed cost of opening facilities
@constraint(model_ext, [i in 1:m, j in 1:n],
    x[i, j] <= y[j]
);

In [37]:
optimize!(model_ext)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 27109.5 - 0.80 seconds
Cgl0004I processed model has 6120 rows, 6050 columns (6050 integer (6050 of which binary)) and 18000 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 197 integers unsatisfied sum - 98.5
Cbc0038I Pass   1: (1.42 seconds) suminf.    0.00000 (0) obj. 56153 iterations 2452
Cbc0038I Solution found of 56153
Cbc0038I Before mini branch and bound, 5769 integers at bound fixed and 0 continuous
Cbc0038I Full problem 6120 rows 6050 columns, reduced to 235 rows 242 columns
Cbc0038I Mini branch and bound improved solution from 56153 to 28673 (1.55 seconds)
Cbc0038I Round again with cutoff of 28515.8
Cbc0038I Reduced cost fixing fixed 30 variables on major pass 2
Cbc0038I Pass   2: (3.00 seconds) suminf.   81.77896 (224) obj. 28515.8 iterations 3169
Cbc0038I Pass   3: (3.14 seco

## Capacitated Facility location

* Each client $i$ has a demand $a_{i}$, and each facility has a capacity $q_{j}$

# Tests

In [49]:
optimizer = GLPK.Optimizer()

A LinQuadOptInterface model with backend:
Prob(Ptr{Nothing} @0x0000000006b21f00)

In [54]:
typeof(optimizer) <: MOI.Utilities.CachingOptimizer

false

In [60]:
factory = with_optimizer(GLPK.Optimizer)

OptimizerFactory(GLPK.Optimizer, (), Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}())